# Error in Floating Point System

Start from a simple example

In [26]:
a64 = 0.01
b64 = 0.2
@show a64 + b64;

a64 + b64 = 0.21000000000000002


In [27]:
a32 = Float32(0.01)
b32 = Float32(0.2)
@show a32 + b32;

a32 + b32 = 0.21000001f0


Here we can see another example

In [24]:
sum_natural = sum(Float32(Float32(1)/Float32(i)^Float32(2)) for i= 1:10000)
sum_opposite = sum(Float32(Float32(1)/Float32(10001 - i)^Float32(2)) for i= 1:10000)
sum_exact = sum(1.0/(i^2) for i in 1:10000)
println("error natural = ", abs(sum_exact - sum_natural))
println("error opposite = ", abs(sum_exact - sum_opposite))

error natural = 0.00010874912467651043
error opposite = 3.0252606197933574e-8
